Course: **Introduction to Artificial Intelligence** \
Lecturer: **Nguyen Thanh An** \
Lab 01: **Uninformed Search**

Students refer to pseudo codes of BFS, DFS, UCS, DLS, and IDS in [this link](https://drive.google.com/file/d/1q2LtrRCfemfiqyhfxNMcVJ3alvLh_pdV/view?usp=share_link) to implement the corresponding classes in TODO 1 - 5. \
Students can add supporting attributes and methods to the five classes as needed.

# Libraries

In [82]:
import os
import heapq

# Graph class

In [83]:
# Directed, weighted graphs
class Graph:
  def __init__(self):
    self.AL = dict() # adjacency list
    self.V = 0
    self.E = 0

  def __str__(self):
    res = 'V: %d, E: %d\n'%(self.V, self.E)
    for u, neighbors in self.AL.items():
      line = '%d: %s\n'%(u, str(neighbors))
      res += line
    return res

  def print(self):
    print(str(self))

  def load_from_file(self, filename):
    '''
        Example input file:
            V E
            u v w
            u v w
            u v w
            ...

        # input.txt
        7 9
        0 1 5
        0 2 6
        1 3 12
        1 4 9
        2 5 5
        3 5 8
        3 6 7
        4 6 4
    '''
    if os.path.exists(filename):
      with open(filename) as g:
        self.V, self.E = [int(it) for it in g.readline().split()]
        for line in g:
          u, v, w = [int(it) for it in line.strip().split()]
          if u not in self.AL:
            self.AL[u] = []
          self.AL[u].append((v, w))

In [84]:
g = Graph()
g.load_from_file('input.txt')
g.print()

V: 7, E: 9
0: [(1, 5), (2, 6)]
1: [(3, 12), (4, 9)]
2: [(5, 5)]
3: [(5, 8), (6, 7)]
4: [(6, 4)]



# Search Strategies

In [85]:
class SearchStrategy:
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    return expanded, path

In [86]:
class BFS(SearchStrategy):
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    if src == dst:
      return expanded, path
    frontier = [src]
    path_cost = 0
    parent = {src:-1}
    flag = False
    while True:
      if len(frontier) == 0:
        break
      u = frontier.pop(0)
      expanded.append(u)
      if u not in g.AL.keys():
        continue
      for v, w in g.AL[u]:
        if v not in expanded and v not in frontier:
          parent[v] = u
          frontier.append(v)
          if v == dst:
            flag = True
            break
      if flag:
        break
    
    if dst in parent.keys():
      u = dst
      while u != -1:
        path = [u] + path
        u = parent[u]
    return expanded, path

In [87]:
class DFS(SearchStrategy):
  def __init__(self):
    self.expanded = []
    self.parent = {}
  
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst

    self.parent[src] = -1
    self.RecurDFS(g, src, dst)
    
    if dst in self.parent.keys():
      u = dst
      while u != -1:
        path = [u] + path
        u = self.parent[u]
    
    expanded = self.expanded
    return expanded, path
  
  def RecurDFS(self, g: Graph, src: int, dst: int) -> tuple:
    self.expanded.append(src)
    if src in g.AL.keys():
      for v, w in g.AL[src]:
        if v not in self.expanded:
          self.parent[v] = src
          self.RecurDFS(g, v, dst)

In [88]:
class UCS(SearchStrategy):
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst

    pq = [(0, src, -1)]
    parent = {src: -1}
    while len(pq) != 0:
      w, u, pu = heapq.heappop(pq)
      parent[u] = pu
      if u == dst:
        break
      if u in expanded:
        continue
      expanded.append(u)
      if u not in g.AL.keys():
        continue
      for v, w_ in g.AL[u]:
        if v in expanded:
          continue
        heapq.heappush(pq, (w + w_, v, u))
    
    if dst in parent.keys():
      u = dst
      while u != -1:
        path = [u] + path
        u = parent[u]
    return expanded, path

In [89]:
class DLS(SearchStrategy):
  def __init__(self, LIM: int):
    self.LIM = LIM
    self.expanded = []
    self.parent = {}
  
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    
    self.parent[src] = -1
    self.RecursiveDLS(g, src, dst, self.LIM)
    expanded = self.expanded
    if dst in self.parent.keys():
      u = dst
      while u != -1:
        path = [u] + path
        u = self.parent[u]
    
    return expanded, path
  
  def RecursiveDLS(self, g: Graph, src: int, dst: int, limit: int) -> tuple:
    self.expanded.append(src)
    if limit == 1:
      return
    if src not in g.AL.keys():
      return
    for v, w in g.AL[src]:
      if v not in self.expanded:
        self.parent[v] = src
        self.RecursiveDLS(g, v, dst, limit - 1)
      

In [90]:
class IDS(SearchStrategy):
  def __init__(self, MAX_LIM: int):
    self.MAX_LIM = MAX_LIM
  
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst

    for lim in range(1, self.MAX_LIM + 1):
      dls = DLS(lim)
      expanded, path = dls.search(g, src, dst)
      if len(path) != 0:
        break
    
    return expanded, path

# Evaluation

In [91]:
bfs = BFS()
dfs = DFS()
ucs = UCS()
dls = DLS(LIM=3)
ids = IDS(MAX_LIM=5)

for stg in [bfs, dfs, ucs, dls, ids]:
  print(stg)
  expanded, path = stg.search(g, 0, g.V-1)
  print(expanded)
  print(path)



[0, 1, 2, 3]
[0, 1, 3, 6]
[0, 1, 3, 5, 6, 4, 2]
[0, 1, 3, 6]
[0, 1, 2, 5, 4, 3]
[0, 1, 4, 6]
[0, 1, 3, 4, 2, 5]
[]
[0, 1, 3, 5, 6, 4, 2]
[0, 1, 3, 6]


# Submission Notice


*   Maintain all cell outputs
*   Download and rename the notebook as **lab01_\<Student ID\>.ipynb**
*   Submit by the deadline
